In [1]:
import os
import json # Added for .ipynb file parsing
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [2]:
load_dotenv()

True

In [3]:
# Get your Groq API key from the environment variables.
groq_api_key = os.getenv("GROQ_API_KEY")

In [4]:
# Check if the API key is available
if not groq_api_key:
    print("Warning: GROQ_API_KEY not found in environment variables or .env file.")
    print("Please ensure you have a .env file in the same directory with GROQ_API_KEY='YOUR_API_KEY'.")
    # You might want to exit or raise an error here if the key is mandatory for your use case.

In [5]:
# --- Initialize the Language Model ---
# Using Groq's Llama3-8b-8192 model for comment generation.
# Added verbose=True for detailed logging during generation.
llm = ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name="meta-llama/llama-4-scout-17b-16e-instruct", verbose=True)

In [6]:
# --- Define the Prompt Template ---
# This template is now language-agnostic.
# It asks the LLM to act as an expert programmer assistant for any language.
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert programmer assistant. Your task is to review the provided code in any language and add comprehensive documentation (e.g., docstrings, function headers, or block comments) and relevant line comments where logic needs clarification. Ensure comments are concise, accurate, and improve code readability without being redundant. Do not change any existing code, only add comments."),
    ("user", "Here is the code:\n```\n{code}\n```\nPlease return the commented code."), # Removed specific language hint like 'python'
])

In [7]:
# --- Create the Langchain Pipeline ---
# This chain combines the prompt, the language model, and an output parser.
chain = prompt | llm | StrOutputParser()

In [8]:
def generate_code_comments(code_snippet: str) -> str:
    """
    Generates documentation and line comments for a given code snippet
    in any programming language using a Langchain pipeline with a Groq model.

    Args:
        code_snippet (str): The raw code string to be commented.

    Returns:
        str: The code with generated comments.
    """
    if not code_snippet.strip():
        return "Please provide some code to generate comments."

    if not groq_api_key:
        return "Error: Groq API key is not set. Cannot generate comments. Check your .env file."

    try:
        print("Generating comments... This might take a moment using Groq.")
        # Invoke the Langchain chain with the user's code
        commented_code = chain.invoke({"code": code_snippet})
        return commented_code
    except Exception as e:
        return f"An error occurred during comment generation: {e}"

In [9]:
def read_py_file(file_path: str) -> str:
    """
    Reads content from a .py file.

    Args:
        file_path (str): The path to the .py file.

    Returns:
        str: The content of the file.
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

def read_ipynb_file(file_path: str) -> str:
    """
    Reads content from a .ipynb (Jupyter Notebook) file.
    Extracts and concatenates code from all code cells.

    Args:
        file_path (str): The path to the .ipynb file.

    Returns:
        str: Concatenated code from all code cells.
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        notebook_content = json.load(f)

    full_code = []
    for cell in notebook_content.get('cells', []):
        if cell.get('cell_type') == 'code':
            # 'source' is a list of lines for each cell
            full_code.extend(cell.get('source', []))
    return "".join(full_code)

In [10]:
# --- Main execution block ---
if __name__ == "__main__":
    print("Welcome to the General Code Comment Generator (using Groq and .env)!")
    print("How would you like to provide the code?")
    print("1. Enter code directly")
    print("2. Provide a file path (.py or .ipynb)")

    choice = input("Enter your choice (1 or 2): ").strip()
    my_code = ""

    if choice == '1':
        print("\nPlease enter your code below (any language). Type 'END' on a new line and press Enter twice to finish.")
        user_code_lines = []
        while True:
            try:
                line = input()
                if line.strip().upper() == 'END':
                    break
                user_code_lines.append(line)
            except EOFError:
                break
        my_code = "\n".join(user_code_lines)
    elif choice == '2':
        file_path = input("Enter the path to your .py or .ipynb file: ").strip()
        if not os.path.exists(file_path):
            print(f"Error: File not found at '{file_path}'")
            exit()

        if file_path.endswith('.py'):
            try:
                my_code = read_py_file(file_path)
                print(f"Successfully loaded code from '{file_path}'")
            except Exception as e:
                print(f"Error reading .py file: {e}")
                exit()
        elif file_path.endswith('.ipynb'):
            try:
                my_code = read_ipynb_file(file_path)
                print(f"Successfully loaded code from '{file_path}'")
            except Exception as e:
                print(f"Error reading .ipynb file: {e}")
                exit()
        else:
            print("Unsupported file type. Please provide a .py or .ipynb file.")
            exit()
    else:
        print("Invalid choice. Please enter 1 or 2.")
        exit()

    if my_code.strip():
        print("\n--- Generating Comments for Your Code ---")
        result = generate_code_comments(my_code)
        print("\n--- Your Commented Code ---")
        print(result)
    else:
        print("No code was provided. Exiting.")

Welcome to the General Code Comment Generator (using Groq and .env)!
How would you like to provide the code?
1. Enter code directly
2. Provide a file path (.py or .ipynb)


Enter your choice (1 or 2):  2
Enter the path to your .py or .ipynb file:  C:\Users\hp\Downloads\G.py


Successfully loaded code from 'C:\Users\hp\Downloads\G.py'

--- Generating Comments for Your Code ---
Generating comments... This might take a moment using Groq.

--- Your Commented Code ---
```python
# Open a file named 'Gireesh.txt' in append mode ('a') to add content without overwriting existing data
file = open('Gireesh.txt', 'a')

# Write a greeting message to the file and store the number of bytes written
content = file.write("Hello, Gireesh!\n")  # Write returns the number of bytes written

# Write another line to the file and store the number of bytes written
content = file.write("This is a New line.\n")  # Overwrites previous content variable

# Close the file to free up system resources
file.close()
```

However, I would suggest a more Pythonic way to handle file operations using a `with` statement, which automatically closes the file:

```python
# Open a file named 'Gireesh.txt' in append mode ('a') using a with statement
with open('Gireesh.txt', 'a') as file:
    # Write a 